## import

In [1]:
import os
import random
import time
import json
import warnings 
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from utils import *
import cv2
from sklearn.model_selection import StratifiedKFold

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import zipfile

# Pretrained Model
import segmentation_models_pytorch as smp

# torchvision Models
import torchvision
from torchvision import models
from torchvision.models.segmentation.deeplabv3 import DeepLabHead

# 전처리를 위한 라이브러리
from torch.utils.data.sampler import WeightedRandomSampler
from pycocotools.coco import COCO
import torchvision
import torchvision.transforms as transforms

import albumentations as A
from albumentations.pytorch import ToTensorV2

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

plt.rcParams['axes.grid'] = False

print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))

print(torch.cuda.get_device_name(0))
print(torch.cuda.device_count())

device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 사용 가능 여부에 따라 device 정보 저장
%matplotlib inline

pytorch version: 1.5.0+cu101
GPU 사용 가능 여부: True
Tesla V100-PCIE-32GB
1


## 하이퍼파라미터 세팅 및 seed 고정

In [2]:
batch_size = 20  # Mini-batch size
num_epochs = 40
learning_rate = 1e-4

In [3]:
# seed 고정
random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
# torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

## Custom Dataset

In [4]:
class TrashDataset(Dataset):
    def __init__(self, df, root='../input/data/', mode="train", transform=None):
        self.df = df.reset_index(drop=True).copy()
        self.mode = mode
        self.transform = transform
        self.root = root
        self.mask_label=[]
        mask_path = self.root + self.df['masks']
        for m_path in tqdm(mask_path.tolist()):
            masks = cv2.imread(m_path, cv2.IMREAD_GRAYSCALE).astype(np.float32)
            self.fold_label(masks)
    
    def fold_label(self, masks):
        len(list(np.unique(masks)))
        if 10.0 in list(np.unique(masks)): self.mask_label.append(0)
        elif 1.0 in list(np.unique(masks)): self.mask_label.append(1)
        elif 11.0 in list(np.unique(masks)): self.mask_label.append(2)
        elif 6.0 in list(np.unique(masks)): self.mask_label.append(3)
        elif 5.0 in list(np.unique(masks)): self.mask_label.append(4)
        elif 4.0 in list(np.unique(masks)): self.mask_label.append(5)
        elif 8.0 in list(np.unique(masks)): self.mask_label.append(6)
        elif 7.0 in list(np.unique(masks)): self.mask_label.append(7)
        elif 2.0 in list(np.unique(masks)): self.mask_label.append(8)
        elif 3.0 in list(np.unique(masks)): self.mask_label.append(9)
        elif 9.0 in list(np.unique(masks)): self.mask_label.append(10)
        else: self.mask_label.append(11)  
            
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_path = self.root + self.df.iloc[idx]['filepath']
        imgs = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        
        if self.mode=="train" or self.mode=="val":
            mask_path = self.root + self.df.iloc[idx]['masks']
            masks = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE).astype(np.float32)
            transformed = self.transform(image=imgs, mask=masks)
            imgs = transformed["image"]
            masks = transformed["mask"]
            return imgs, masks
        
        elif self.mode == "test":
            transformed = self.transform(image=imgs)
            imgs = transformed["image"]
            return imgs

In [5]:
# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))

train_transform = A.Compose([
                            A.Rotate(border_mode=1, p=0.5),
                            A.ShiftScaleRotate(border_mode=1, p=0.5),
                            A.HorizontalFlip(p=0.5),
                            A.VerticalFlip(p=0.5),
                            A.Cutout(),
                            A.Normalize(
                                mean=(0.485, 0.456, 0.406),
                                std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0
                            ),
                            ToTensorV2(),
                            ])

test_transform = A.Compose([
                            A.Normalize(
                                mean=(0.485, 0.456, 0.406),
                                std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0
                            ),
                            ToTensorV2(),
                           ])

In [6]:
df = pd.read_csv("../input/data/train.csv")

additional_df = pd.read_csv("../input/data/test.csv")
df = pd.concat([df, additional_df], ignore_index=True)

In [7]:
train_dataset = TrashDataset(df, mode="train", transform=train_transform)

  0%|          | 0/4109 [00:00<?, ?it/s]

## Class Score

In [8]:
def class_score(avrg_class_IoU):
    # Class Score
    class_name=['BG','UNK','General Trash','Paper','Paper pack','Metal','Glass','Plastic','Styrofoam','Plastic Bag','Battery','Clothing']
    print('-'*80)
    print('Validation Class Pred mIoU Score')
    for idx, class_score in enumerate(avrg_class_IoU):
        print('[{}] mIoU : [{:.4f}]'.format(class_name[idx],class_score))
    print('-'*80) 

## CutMix

In [11]:
def rand_bbox(W, H, lam):
    cut_rat = torch.sqrt(1.0 - lam)
    cut_w = (W * cut_rat).type(torch.long)
    cut_h = (H * cut_rat).type(torch.long)
    # uniform
    cx = torch.randint(W, (1,)).to(device)
    cy = torch.randint(H, (1,)).to(device)
    x1 = torch.clamp(cx - cut_w // 2, 0, W)
    y1 = torch.clamp(cy - cut_h // 2, 0, H)
    x2 = torch.clamp(cx + cut_w // 2, 0, W)
    y2 = torch.clamp(cy + cut_h // 2, 0, H)
    return x1, y1, x2, y2

In [12]:
def cutmix_data(x, y, alpha=1.0, p=0.3):
    # x-> img 
    # y-> mask
    if np.random.random() > p:
        return x, y
    W, H = x.size(2), x.size(3)
    shuffle = torch.randperm(x.size(0)).to(device)
    cutmix_x = x
    cutmix_y = y

    lam = torch.distributions.beta.Beta(alpha, alpha).sample().to(device)
    x1, y1, x2, y2 = rand_bbox(W, H, lam)
    cutmix_x[:, :, x1:x2, y1:y2] = x[shuffle, :, x1:x2, y1:y2]
    cutmix_y[:, x1:x2, y1:y2] = y[shuffle, x1:x2, y1:y2]
    # Adjust lambda to match pixel ratio
    #lam = 1 - ((x2 - x1) * (y2 - y1) / float(W * H)).item()
    return cutmix_x, cutmix_y

## train, validation, test 함수 정의

In [13]:
def train(fold, num_epochs, model, data_loader, val_loader, criterion, optimizer, scheduler, device):
    print('-'*80)
    print(f'Fold : [{fold}] Start training..')
    print('-'*80)
    early_stop=EarlyStopping(patience=5,path='./saved/'+str(fold)+'_checkpoint.pt')

    for epoch in range(num_epochs):
        train_loss=[]
        model.train()
        for step, (images, masks) in enumerate(tqdm(data_loader)):
            images = torch.stack(images)       # (batch, channel, height, width)
            masks = torch.stack(masks).long()  # (batch, channel, height, width)

            # gpu 연산을 위해 device 할당
            images, masks = images.to(device), masks.to(device)

            #Cut-Mix
            images, masks = cutmix_data(images, masks)

            # inference
            outputs = model(images)

            # loss 계산
            loss = criterion(outputs, masks)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

            #Cosine Annealing Scheduler
            #scheduler.step(epoch + step / len(data_loader))

        # validation 주기에 따른 loss 출력 및 best model 저장
        avrg_loss, avrg_IoU, avrg_class_IoU, pb_IoU = validation(model, val_loader, device)
        print('Epoch [{}/{}], Train Loss: {:.4f} Vali Loss: {:.4f}, Vali mIoU: {:.4f}, Vali Public mIoU: {:.4f}'.format(epoch+1, num_epochs, np.mean(train_loss), avrg_loss, avrg_IoU, pb_IoU))
        # Reduce Scheduler
        scheduler.step(avrg_IoU)
        # Class Score
        #class_score(avrg_class_IoU)
        # Save
        early_stop(avrg_IoU,model)
        if early_stop.early_stop:
            print('Stop Training.....')
            break

In [14]:
def validation(model, data_loader, device):
    criterion = nn.CrossEntropyLoss().to(device)
    n_class=12
    hist = np.zeros((n_class, n_class))
    print('Start validation')
    model.eval()
    with torch.no_grad():
        total_loss = 0
        cnt = 0
        pb_mIoU_list = []
        for step, (images, masks) in enumerate(tqdm(data_loader)):
            images = torch.stack(images)       # (batch, channel, height, width)
            masks = torch.stack(masks).long()  # (batch, channel, height, width)

            images, masks = images.to(device), masks.to(device)        

            outputs = model(images)
            loss = criterion(outputs, masks)
            total_loss += loss
            cnt += 1
            
            outputs = torch.argmax(outputs, dim=1).detach().cpu().numpy()
            
            hist = add_hist(hist, masks.detach().cpu().numpy(), outputs, n_class=n_class)
            pb_mIoU = public_label_accuracy_score(masks.detach().cpu().numpy(), outputs, n_class=12)
            pb_mIoU_list.append(pb_mIoU)
        
        vali_mIoU, vali_class_mIoU = label_accuracy_score(hist)
        avrg_loss = total_loss / cnt

    return avrg_loss, vali_mIoU, vali_class_mIoU, np.mean(pb_mIoU_list)

## 모델 생성 및 Loss function, Optimizer 정의

In [ ]:
folds=StratifiedKFold(n_splits=5,shuffle=True)
for current_fold,(train_idx, vali_idx) in enumerate(folds.split(train_dataset,train_dataset.mask_label)):
    model = smp.DeepLabV3Plus('resnext50_32x4d', encoder_weights="swsl", classes=12)
    model.eval()
    model.to(device)

    train_data=torch.utils.data.Subset(train_dataset,train_idx)
    vali_data=torch.utils.data.Subset(train_dataset,vali_idx)

    train_loader=DataLoader(train_data,batch_size=batch_size,shuffle=True,collate_fn=collate_fn,num_workers=0)
    val_loader=DataLoader(vali_data,batch_size=1,shuffle=False,collate_fn=collate_fn,num_workers=0)

    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(params = model.parameters(), lr = learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1,threshold_mode='abs',min_lr=1e-8, verbose=True)
    #scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=20, eta_max=learning_rate,  T_up=2, gamma=0.5)
    
    train(current_fold, num_epochs, model, train_loader, val_loader, criterion, optimizer, scheduler, device)

--------------------------------------------------------------------------------
Fold : [0] Start training..
--------------------------------------------------------------------------------


  0%|          | 0/165 [00:00<?, ?it/s]

Start validation


  0%|          | 0/822 [00:00<?, ?it/s]

Epoch [1/40], Train Loss: 0.9458 Vali Loss: 0.4891, Vali mIoU: 0.3748, Vali Public mIoU: 0.4899


...

  0%|          | 0/165 [00:00<?, ?it/s]

Start validation


  0%|          | 0/822 [00:00<?, ?it/s]

Epoch [4/40], Train Loss: 0.2955 Vali Loss: 0.3162, Vali mIoU: 0.4567, Vali Public mIoU: 0.5569


  0%|          | 0/165 [00:00<?, ?it/s]